In [1]:
from docplex.mp.model import Model
import itertools

def input_user_data():
    longueur_barre = 6000
    longueurs = []
    demandes = {}
    n = int(input("Combien de longueurs différentes voulez-vous couper ? "))
    for _ in range(n):
        l = int(input("Longueur de pièce (en mm) : "))
        q = int(input(f"Quantité demandée pour {l} mm : "))
        longueurs.append(l)
        demandes[l] = q
    return longueur_barre, longueurs, demandes

def generate_patterns(longueurs, longueur_max):
    patterns = []
    for r in range(1, len(longueurs)+5):  # +5 pour permettre des combinaisons plus longues
        for comb in itertools.combinations_with_replacement(longueurs, r):
            if sum(comb) <= longueur_max:
                pattern = {l: comb.count(l) for l in longueurs}
                if pattern not in patterns:
                    patterns.append(pattern)
    return patterns

def solve_cutting_stock(longueur_barre, longueurs, demandes):
    patterns = generate_patterns(longueurs, longueur_barre)
    print(f"{len(patterns)} modèles de découpe générés.")
    
    mdl = Model("cutting_stock")
    x = {i: mdl.integer_var(name=f"x_{i}") for i in range(len(patterns))}

    # Contraintes de satisfaction de la demande
    for l in longueurs:
        mdl.add_constraint(mdl.sum(x[i] * patterns[i][l] for i in range(len(patterns))) >= demandes[l])

    mdl.minimize(mdl.sum(x[i] for i in range(len(patterns))))

    solution = mdl.solve(log_output=True)

    if solution:
        print("\n--- SOLUTION OPTIMALE ---")
        total_barres = 0
        for i in range(len(patterns)):
            if x[i].solution_value >= 1:
                print(f"Utiliser {int(x[i].solution_value)} fois le modèle {patterns[i]}")
                total_barres += int(x[i].solution_value)
        print(f"\nNombre total de barres nécessaires : {total_barres}")
    else:
        print("Aucune solution trouvée.")

# ============================
# Programme principal
# ============================

if __name__ == "__main__":
    longueur_barre, longueurs, demandes = input_user_data()
    solve_cutting_stock(longueur_barre, longueurs, demandes)


Combien de longueurs différentes voulez-vous couper ?  3
Longueur de pièce (en mm) :  2500
Quantité demandée pour 2500 mm :  2
Longueur de pièce (en mm) :  390
Quantité demandée pour 390 mm :  2
Longueur de pièce (en mm) :  626
Quantité demandée pour 626 mm :  1


65 modèles de découpe générés.
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 5.000000 after 0.05 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 0 rows and 54 columns.
MIP Presolve added 3 rows and 3 columns.
MIP Presolve modified 59 coefficients.
Reduced MIP has 6 rows, 14 columns, and 29 nonzeros.
Reduced MIP has 1 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.18 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 3 rows and 3 columns.
MIP Presolve added 3 rows and 3 columns.
Reduced MIP has 6 rows, 14 columns, and 29 nonzeros.
Reduced MIP has 1 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, usin

In [1]:
from docplex.mp.model import Model
import itertools

# -----------------------
# DONNEES DU PROBLEME
# -----------------------
L = 6000  # Longueur barre brute

# Longueurs et quantités demandées
demande = {2500: 2, 626: 1, 470: 2}

longueurs = list(demande.keys())
quantites = list(demande.values())

# -----------------------
# GENERATION DES PATTERNS
# -----------------------

def generer_patterns(longueurs, L):
    patterns = []
    max_coupes = [L // l for l in longueurs]
    
    # Essayer toutes les combinaisons possibles
    for n_coupes in itertools.product(*[range(max_coupes[i]+1) for i in range(len(longueurs))]):
        longueur_totale = sum(n_coupes[i] * longueurs[i] for i in range(len(longueurs)))
        if 0 < longueur_totale <= L:
            patterns.append((n_coupes, L - longueur_totale))
    return patterns

patterns = generer_patterns(longueurs, L)

print(f"Nombre de patterns générés: {len(patterns)}")

# -----------------------
# OPTIMISATION
# -----------------------
mdl = Model("decoupage")

# Variable x_p = nombre de barres utilisées selon pattern p
x = [mdl.integer_var(name=f"x_{i}") for i in range(len(patterns))]

# Contraintes : satisfaire la demande
for j, l in enumerate(longueurs):
    mdl.add_constraint(mdl.sum(x[i] * patterns[i][0][j] for i in range(len(patterns))) >= demande[l])

# Fonction objectif : minimiser la somme des restes
mdl.minimize(mdl.sum(x[i] * patterns[i][1] for i in range(len(patterns))))

# Résolution
solution = mdl.solve()

if solution:
    print("\n--- Résultat optimal ---")
    total_barres = sum(solution[x[i]] for i in range(len(patterns)))
    total_reste = sum(solution[x[i]] * patterns[i][1] for i in range(len(patterns)))
    
    print(f"Nombre total de barres utilisées: {total_barres}")
    print(f"Reste total cumulé: {total_reste} mm")
    
    for i in range(len(patterns)):
        if solution[x[i]] > 0:
            print(f"Utiliser {int(solution[x[i]])} barre(s) avec pattern {patterns[i][0]} (reste: {patterns[i][1]} mm)")
else:
    print("Pas de solution trouvée")


Nombre de patterns générés: 104

--- Résultat optimal ---
Nombre total de barres utilisées: 2.0
Reste total cumulé: 108.0 mm
Utiliser 1 barre(s) avec pattern (0, 2, 10) (reste: 48 mm)
Utiliser 1 barre(s) avec pattern (2, 0, 2) (reste: 60 mm)


In [3]:
# backend_decoupe.py

from docplex.mp.model import Model
import itertools

def generer_patterns(longueurs, L):
    patterns = []
    max_coupes = [L // l for l in longueurs]
    
    for n_coupes in itertools.product(*[range(max_coupes[i]+1) for i in range(len(longueurs))]):
        longueur_totale = sum(n_coupes[i] * longueurs[i] for i in range(len(longueurs)))
        if 0 < longueur_totale <= L:
            patterns.append((n_coupes, L - longueur_totale))
    return patterns

def optimiser_decoupe(longueurs, quantites, L=6000):
    patterns = generer_patterns(longueurs, L)
    
    mdl = Model("decoupe")
    x = [mdl.integer_var(name=f"x_{i}") for i in range(len(patterns))]
    
    for j, l in enumerate(longueurs):
        mdl.add_constraint(mdl.sum(x[i] * patterns[i][0][j] for i in range(len(patterns))) >= quantites[j])
    
    mdl.minimize(mdl.sum(x[i] * patterns[i][1] for i in range(len(patterns))))
    
    solution = mdl.solve()
    
    resultats = []
    if solution:
        for i in range(len(patterns)):
            if solution[x[i]] > 0:
                resultats.append({
                    'pattern': patterns[i][0],
                    'reste': patterns[i][1],
                    'quantite': int(solution[x[i]])
                })
    return resultats


In [10]:
pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 2.7 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 24.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
# app.py

import streamlit as st
#from backend_decoupe import optimiser_decoupe
import plotly.graph_objects as go

st.title("Optimisation de découpe de barres")

st.header("Entrez les longueurs et quantités demandées :")
n = st.number_input("Nombre de types de longueurs :", min_value=1, step=1)

longueurs = []
quantites = []
for i in range(n):
    col1, col2 = st.columns(2)
    with col1:
        l = st.number_input(f"Longueur {i+1} (mm):", min_value=1)
    with col2:
        q = st.number_input(f"Quantité {i+1}:", min_value=1, step=1)
    longueurs.append(int(l))
    quantites.append(int(q))

if st.button("Calculer"):
    resultats = optimiser_decoupe(longueurs, quantites)
    
    st.success("Optimisation terminée !")
    for idx, res in enumerate(resultats):
        st.write(f"Barre {idx+1} - {res['quantite']} fois")
        st.write(f"Pattern: {res['pattern']}, Reste: {res['reste']} mm")
        
        # Visualisation plotly
        for k in range(res['quantite']):
            fig = go.Figure()
            pos = 0
            colors = ['blue', 'green', 'red', 'orange', 'purple', 'cyan']
            for j, count in enumerate(res['pattern']):
                for c in range(count):
                    fig.add_trace(go.Bar(
                        x=[1], y=[longueurs[j]],
                        base=pos,
                        width=[0.5],
                        marker_color=colors[j % len(colors)],
                        name=f"{longueurs[j]} mm"
                    ))
                    pos += longueurs[j]
            fig.add_trace(go.Bar(
                x=[1], y=[res['reste']],
                base=pos,
                width=[0.5],
                marker_color='lightgray',
                name='Chute'
            ))
            fig.update_layout(barmode='stack', title=f"Visualisation découpe barre {idx+1} - instance {k+1}", yaxis_title="Longueur (mm)")
            st.plotly_chart(fig)


2025-05-05 17:04:56.865 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-05 17:04:57.107 
  command:

    streamlit run /home/issiaka/BSS_code/env310bss/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-05-05 17:04:57.109 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-05 17:04:57.110 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-05 17:04:57.112 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-05 17:04:57.114 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-05 17:04:57.115 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-05 17:04:57.117 Thre

In [ ]:
!streamlit run app.py


      👋 Welcome to Streamlit!

      If you’d like to receive helpful onboarding emails, news, offers, promotions,
      and the occasional swag, please enter your email address below. Otherwise,
      leave this field blank.

      Email:  